In [1]:
import torch
from model import LayoutLMT5
from metrics import compute_f1, compute_exact

PL version: 1.0.7
pytorch version: 1.7.0+cu101
Transformers version: 4.1.1


# T5 Base (without LayoutLM)

In [2]:
model = LayoutLMT5.load_from_checkpoint("models/slower/slower-epoch=5.ckpt", strict=False).eval().cuda()
model.hparams.bs = 10
val_dataloader = iter(model.test_dataloader())

Initializing T5...


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


test DocVQA folder data/raw/test tokenizer PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra

In [3]:
batch = next(val_dataloader)
cuda_batch = {}
for k, v in batch.items():
    if k in ["document", "target_text", "input_text"]:
        cuda_batch[k] = v
    else:
        cuda_batch[k] = v.cuda()  
batch = cuda_batch
tokenizer = model.tokenizer

with torch.no_grad():
    output = model(batch)

output_text = [tokenizer.decode(out, skip_special_tokens=True) for out in output]
tgt_text = batch["target_text"]
question = batch["input_text"]
for i in range(10):
    print(f"input_text: {question[i]}\n")
    print(f"prediction: {output_text[i]}\ntarget: {tgt_text[i]}\nF1: {compute_f1(output_text[i], tgt_text[i])}, Exact: {compute_exact(output_text[i], tgt_text[i])}\n")

input_text: question: What is the dividend payout in 2012? input: ITC Limited . Report and Accounts 2012 Net Worth Crores * Crores CAGR 16% CAGR 19% 18782 2607 1121 1996 2001 2006 2007 2008 2009 2010 2011 2012 1996 2001 2006 2007 2008 2009 2010 2011 2012 Crores CAGR 19% CAGR 30% 1996 2001 2006 2007 2008 2009 : 2010 2011 2012 1996 2 2006 2007 2008 2009 2010 2011 2012 Inclusive of Dividend Distribution Tax 2010-Special Centenary Dividend 2011-Special Dividend on 100th AGM Earnings And Dividend Per Share (Adjusted' 8/ 40% 30% 6 20% 10% 2 0% -10% L. 16 years 10 years 5 years 3 years 1996 2001 2006 2007 2008 2009 2010 2011 2012 1 year BSE FMCG Index Earning Per Share Dividend Per Share Special Dividend Per Share BSE Sensex ITC BSE FMCG Index commenced from 2000 Dividend Per Share and Dividend Payout for 2012 is based on the Board's recommendation to Members ^ Refer Page 176 for details of Adjusted Earnings/ Dividends/Net Worth per share Source: https://www.industrydocuments.ucsf.edu/docs/rn

# LayoutLMT5 model

In [13]:
model = LayoutLMT5.load_from_checkpoint("models/frozen_llm/frozen_llm-epoch=8-val_loss=0.00-val_extact_match=0.00-val_f1=0.00.ckpt", strict=False).eval().cuda()
model.hparams.bs = 10
val_dataloader = iter(model.val_dataloader())

Initializing LayoutLM...


Freezing LayoutLM...: 203it [00:00, 88059.13it/s]


Initializing T5...


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


val DocVQA folder data/raw/val tokenizer PreTrainedTokenizer(name_or_path='microsoft/layoutlm-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}) transform None seq_len 512 no_image True


In [14]:
batch = next(val_dataloader)
cuda_batch = {}
for k, v in batch.items():
    if k in ["document", "target_text", "input_text"]:
        cuda_batch[k] = v
    else:
        cuda_batch[k] = v.cuda()  
batch = cuda_batch
tokenizer = model.tokenizer

with torch.no_grad():
    output = model(batch)

output_text = [tokenizer.decode(out, skip_special_tokens=True) for out in output]
tgt_text = batch["target_text"]
for i in range(10):
    print(f"input_text: {question[i]}\n")
    print(f"prediction: {output_text[i]}\ntarget: {tgt_text[i]}\nF1: {compute_f1(output_text[i], tgt_text[i])}, Exact: {compute_exact(output_text[i], tgt_text[i])}\n")

input_text: question: What is the page no mentioned in this document? input: Procter & Gamble: Diapers In an effort to regain branded market share, Procter & Gamble is returning to a brand equity strategy. 1923 Initiatives . Increase advertising -Performance, Value Positionings -Major TV Campaign . Roll-out disposable training pants . Test 50% thinner diapers (same as KC new product) 51336 9526 16 

prediction: $ 1, 000
target: 0.28
F1: 0, Exact: 0

input_text: question: Who supported the workshop? input: MEHARRY MEDICAL COLLEGE CENTER FOR NUTRITION ARR MEDICAL COL 3931 1 NASHVILLE TENNESSEE FIRST NUTRITION WORKSHOP "THE IMPACT OF NUTRITION ON HEALTH AND DISEASE IN BLACKS AND OTHER MINORITIES" OCTOBER 28-30, 1987 LECTURE HALL #3 S.S. KRESGE LEARNING RESOURCES CENTER SUPPORTED BY: GENERAL FOODS FUND, INC. 

prediction: university of california
target: university of california
F1: 1.0, Exact: 1

input_text: question: What is written in the picture of title 'John Players Jeans'? input: FM

# Layout Embeddings T5

In [4]:
model = LayoutLMT5.load_from_checkpoint("models/llmemb_32_6/llmemb_32_6-epoch=28-val_exact_match=0.49-val_f1=0.58.ckpt", strict=False).eval().cuda()
model.hparams.bs = 10
val_dataloader = iter(model.test_dataloader())

Initializing T5...


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initializing layoutlm embeddings
test DocVQA folder data/raw/test tokenizer PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_

In [5]:
batch = next(val_dataloader)
cuda_batch = {}
for k, v in batch.items():
    if k in ["document", "target_text", "input_text"]:
        cuda_batch[k] = v
    else:
        cuda_batch[k] = v.cuda()  
batch = cuda_batch
tokenizer = model.tokenizer

with torch.no_grad():
    output = model(batch)

output_text = [tokenizer.decode(out, skip_special_tokens=True) for out in output]
tgt_text = batch["target_text"]
question = batch["input_text"]
for i in range(10):
    print(f"input_text: {question[i]}\n")
    print(f"prediction: {output_text[i]}\ntarget: {tgt_text[i]}\nF1: {compute_f1(output_text[i], tgt_text[i])}, Exact: {compute_exact(output_text[i], tgt_text[i])}\n")

input_text: question: What is the dividend payout in 2012? input: ITC Limited . Report and Accounts 2012 Net Worth Crores * Crores CAGR 16% CAGR 19% 18782 2607 1121 1996 2001 2006 2007 2008 2009 2010 2011 2012 1996 2001 2006 2007 2008 2009 2010 2011 2012 Crores CAGR 19% CAGR 30% 1996 2001 2006 2007 2008 2009 : 2010 2011 2012 1996 2 2006 2007 2008 2009 2010 2011 2012 Inclusive of Dividend Distribution Tax 2010-Special Centenary Dividend 2011-Special Dividend on 100th AGM Earnings And Dividend Per Share (Adjusted' 8/ 40% 30% 6 20% 10% 2 0% -10% L. 16 years 10 years 5 years 3 years 1996 2001 2006 2007 2008 2009 2010 2011 2012 1 year BSE FMCG Index Earning Per Share Dividend Per Share Special Dividend Per Share BSE Sensex ITC BSE FMCG Index commenced from 2000 Dividend Per Share and Dividend Payout for 2012 is based on the Board's recommendation to Members ^ Refer Page 176 for details of Adjusted Earnings/ Dividends/Net Worth per share Source: https://www.industrydocuments.ucsf.edu/docs/rn